In [3]:
import json
import os
from PIL import Image
import torch
from torch.utils.data import Dataset

In [ ]:
# COCO JSON 파일 로드
type = 'val'
data_dir = f"../../../Data/door_image_set/{type}"
with open(f"{data_dir}/{type}_coco_annotations.json", "r") as f:
    coco_data = json.load(f)

# 데이터 확인
print("Images:", len(coco_data["images"]))
print("Annotations:", len(coco_data["annotations"]))
print("Categories:", len(coco_data["categories"]))

In [ ]:
class CustomCocoDataset(Dataset):
    def __init__(self, image_dir, annotation_file, transforms=None):
        self.image_dir = image_dir
        self.transforms = transforms

        # COCO JSON 로드
        with open(annotation_file, "r") as f:
            self.coco_data = json.load(f)

        self.images = self.coco_data["images"]
        self.annotations = self.coco_data["annotations"]
        self.categories = self.coco_data["categories"]

        # 이미지 ID와 주석 매핑
        self.image_id_to_annotations = {img["id"]: [] for img in self.images}
        for ann in self.annotations:
            self.image_id_to_annotations[ann["image_id"]].append(ann)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        # 이미지 로드
        image_info = self.images[idx]
        image_path = os.path.join(self.image_dir, image_info["file_name"])
        image = Image.open(image_path).convert("RGB")

        # 바운딩 박스 및 라벨 로드
        annotations = self.image_id_to_annotations[image_info["id"]]
        boxes = []
        labels = []
        for ann in annotations:
            boxes.append(ann["bbox"])
            labels.append(ann["category_id"])

        # Tensor로 변환
        boxes = torch.tensor(boxes, dtype=torch.float32)
        labels = torch.tensor(labels, dtype=torch.int64)

        target = {
            "boxes": boxes,
            "labels": labels,
            "image_id": torch.tensor([image_info["id"]]),
        }

        if self.transforms:
            image, target = self.transforms(image, target)

        return image, target

# 데이터셋 초기화
train_dataset = CustomCocoDataset(
    image_dir=f"{data_dir}/images",
    annotation_file=f"{data_dir}/{type}_coco_annotations.json",
    transforms=None
)

# 데이터셋 확인
print(len(train_dataset))

In [8]:
# Faster R-CNN 파인튜닝
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torch.utils.data import DataLoader
import torchvision
import torch

In [ ]:
# Pre-trained 모델 불러오기
model = fasterrcnn_resnet50_fpn(pretrained=True)

In [10]:
# 데이터셋 정의 (CustomCocoDataset 사용)
train_dataset = CustomCocoDataset(
    image_dir=f"{data_dir}/images",
    annotation_file=f"{data_dir}/{type}_coco_annotations.json"
)
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=4)

In [11]:
# 모델 초기화 및 출력 레이어 수정
model = fasterrcnn_resnet50_fpn(pretrained=True)

In [12]:
# num_classes 수정: 1 (door) + 1 (background)
num_classes = 2
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(
    in_features, num_classes
)

In [ ]:
# GPU로 이동
# model.to("cuda")
model.to("cpu")

In [14]:
# 옵티마이저 설정
optimizer = torch.optim.SGD(model.parameters(), lr=0.005, momentum=0.9)

In [ ]:
# 학습 루프
for epoch in range(10):
    model.train()
    for images, targets in train_loader:
        # images = [img.to("cuda") for img in images]
        # targets = [{k: v.to("cuda") for k, v in t.items()} for t in targets]

        # CPU 사용
        images = [img.to("cpu") for img in images]
        targets = [{k: v.to("cpu") for k, v in t.items()} for t in targets]

        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

    print(f"Epoch {epoch}, Loss: {losses.item()}")

In [ ]:
data_dir

In [ ]:
from PIL import Image

# COCO JSON 로드
annotation_file = f"{data_dir}/{type}_coco_annotations.json"
image_dir = f"{data_dir}/images"

with open(annotation_file, "r") as f:
    coco_data = json.load(f)

# 이미지 ID와 이미지 파일 검증
valid_image_ids = set(img["id"] for img in coco_data["images"])

for ann in coco_data["annotations"]:
    # image_id 확인
    if ann["image_id"] not in valid_image_ids:
        print(f"Invalid image_id: {ann['image_id']} in annotation {ann}")

    # Bounding Box 확인
    bbox = ann["bbox"]
    if len(bbox) != 4 or any(coord < 0 for coord in bbox):
        print(f"Invalid bbox: {bbox} in annotation {ann}")

    # 이미지 크기와 Bounding Box 확인
    image_info = next((img for img in coco_data["images"] if img["id"] == ann["image_id"]), None)
    if image_info:
        width, height = image_info["width"], image_info["height"]
        x, y, w, h = bbox
        if x + w > width or y + h > height:
            print(f"Bbox exceeds image bounds: {bbox} in annotation {ann}")


In [18]:
for img_info in coco_data["images"]:
    img_path = os.path.join(image_dir, img_info["file_name"])
    try:
        with Image.open(img_path) as img:
            img.verify()
    except Exception as e:
        print(f"Invalid image file: {img_path}, Error: {e}")

In [ ]:
valid_annotations = []
for ann in coco_data["annotations"]:
    bbox = ann["bbox"]
    if len(bbox) == 4 and all(coord >= 0 for coord in bbox):
        valid_annotations.append(ann)

coco_data["annotations"] = valid_annotations

# 수정된 JSON 저장
with open("val_labels_fixed.json", "w") as f:
    json.dump(coco_data, f, indent=4)


In [ ]:
from PIL import Image, ImageDraw, ImageFont

# 경로 설정
annotation_file = f"{data_dir}/{type}_coco_annotations.json"  # COCO JSON 파일
image_dir = f"{data_dir}/images"  # 원본 이미지 경로
output_dir = f"visualized_images"  # 시각화된 이미지 저장 경로
os.makedirs(output_dir, exist_ok=True)

# COCO JSON 파일 로드
with open(annotation_file, "r") as f:
    coco_data = json.load(f)

# 이미지 ID와 파일명 매핑
image_id_to_filename = {img["id"]: img["file_name"] for img in coco_data["images"]}

# Bounding Box를 이미지에 그리기
for ann in coco_data["annotations"]:
    image_id = ann["image_id"]
    bbox = ann["bbox"]
    category_id = ann["category_id"]

    # 이미지 파일 경로 확인
    if image_id not in image_id_to_filename:
        print(f"Image ID {image_id} not found in images.")
        continue

    image_file = os.path.join(image_dir, image_id_to_filename[image_id])
    if not os.path.exists(image_file):
        print(f"Image file {image_file} not found.")
        continue

    # 이미지 로드
    try:
        img = Image.open(image_file)
    except Exception as e:
        print(f"Failed to load image: {image_file}, Error: {e}")
        continue

    # 드로잉 객체 생성
    draw = ImageDraw.Draw(img)
    x, y, w, h = bbox
    x1, y1, x2, y2 = x, y, x + w, y + h

    # Bounding Box 그리기
    box_color = (0, 255, 0)  # 초록색
    draw.rectangle([x1, y1, x2, y2], outline=box_color, width=3)

    # 클래스 ID 표시
    text_color = (255, 0, 0)  # 빨간색
    font_size = 20
    try:
        font = ImageFont.truetype("arial.ttf", font_size)
    except IOError:
        font = ImageFont.load_default()  # 기본 폰트 사용
    draw.text((x1, y1 - font_size), f"Class: {category_id}", fill=text_color, font=font)

    # 시각화된 이미지 저장
    output_path = os.path.join(output_dir, image_id_to_filename[image_id])
    img.save(output_path)

print(f"시각화된 이미지가 {output_dir}에 저장되었습니다.")




In [ ]:
# 그려진 바운딩 박스 이상 있음 -> 박스 좌표 다시 그리는거 필요